In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarr

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
import transformers
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

In [ ]:
df = pd.read_csv('training_data_refined.csv')
df['text'] = df.apply(lambda row: row['description'] + " ->: " + row['query'], axis = 1)

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace= True)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df.head(10)

,description,query,text
85,Get the Unicode code point of the first character in the film_name column.,SELECT unicode(film_name) FROM films;,Get the Unicode code point of the first character in the film_name column. ->: SELECT unicode(fi...
35,Retrieve the details of films with release years formatted as 'YYYY-MM-DD'.,"SELECT film_name, strftime('%Y-%m-%d', release_year) AS formatted_release_year FROM films;",Retrieve the details of films with release years formatted as 'YYYY-MM-DD'. ->: SELECT film_name...
26,Retrieve the details of films with a rating higher than the average rating of all films.,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films);,Retrieve the details of films with a rating higher than the average rating of all films. ->: SEL...
47,Retrieve films with release years shifted by 5 years into the future.,"SELECT film_name, release_year + 5 AS future_release_year FROM films;","Retrieve films with release years shifted by 5 years into the future. ->: SELECT film_name, rele..."
31,Retrieve the details of films with a rating higher than the average rating of films watched on J...,"SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE strftime('%Y-%m-%d', wat...",Retrieve the details of films with a rating higher than the average rating of films watched on J...
11,Retrieve the details of films with a specific name.,SELECT * FROM films WHERE film_name = 'The Matrix';,Retrieve the details of films with a specific name. ->: SELECT * FROM films WHERE film_name = 'T...
65,Find films released in the same year as other films.,"SELECT f1.film_name AS film1, f2.film_name AS film2, f1.release_year FROM films f1 JOIN films f2...","Find films released in the same year as other films. ->: SELECT f1.film_name AS film1, f2.film_n..."
28,Retrieve the details of films with a rating higher than the average rating of films released in ...,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = 2022);,Retrieve the details of films with a rating higher than the average rating of films released in ...
69,Display films with ratings formatted to two decimal places.,"SELECT film_name, format(rating, '0.00') AS formatted_rating FROM films;","Display films with ratings formatted to two decimal places. ->: SELECT film_name, format(rating,..."
9,Retrieve the details of films released after a certain year.,SELECT * FROM films WHERE release_year > 2020;,Retrieve the details of films released after a certain year. ->: SELECT * FROM films WHERE relea...


In [ ]:
test_df.head(10)

,description,query,text
44,Retrieve the details of films with their release years converted to Julian day numbers.,"SELECT film_name, julianday(release_year) AS julian_release_year FROM films;",Retrieve the details of films with their release years converted to Julian day numbers. ->: SELE...
53,Show films with ratings ranked in descending order.,"SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;","Show films with ratings ranked in descending order. ->: SELECT film_name, rating, RANK() OVER (O..."
30,Retrieve the details of films with a rating higher than the average rating of films released in ...,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT r...,Retrieve the details of films with a rating higher than the average rating of films released in ...
12,Retrieve the highest rating among all films.,SELECT MAX(rating) FROM films;,Retrieve the highest rating among all films. ->: SELECT MAX(rating) FROM films;
49,Display films with their review lengths calculated.,"SELECT film_name, LENGTH(review) AS review_length FROM films;","Display films with their review lengths calculated. ->: SELECT film_name, LENGTH(review) AS revi..."
0,List all films in the database.,SELECT * FROM films;,List all films in the database. ->: SELECT * FROM films;
64,Retrieve films with release years that are leap years.,SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 40...,Retrieve films with release years that are leap years. ->: SELECT * FROM films WHERE release_yea...
18,Find the average rating of films watched in 2022.,"SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';",Find the average rating of films watched in 2022. ->: SELECT AVG(rating) FROM films WHERE strfti...
10,Retrieve the details of films with a rating within a specific range.,SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;,Retrieve the details of films with a rating within a specific range. ->: SELECT * FROM films WHE...
22,Calculate the average rating of films released in each year.,"SELECT release_year, AVG(rating) FROM films GROUP BY release_year;","Calculate the average rating of films released in each year. ->: SELECT release_year, AVG(rating..."


In [ ]:
train_dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df),
})

In [ ]:
# model_name = "ybelkada/falcon-7b-sharded-bf16"
model_name = "tiiuae/falcon-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/7.16k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
torch.cuda.is_available()

True

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


sequences = pipeline(
   ["“List all films in the database.” ->:","Retrieve the details of films with a rating higher than a certain value. ->:","Retrieve the details of films released after a certain year. ->:"],
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: “List all films in the database.” ->: “List all films in the database. The output is ordered alphabetically, then by release year, by default.”
“List films by release year” ->: “List all films in the database by release year.” The output is ordered alphabetically.
“List films by country” ->: “List all films in the database from a specific country.” The output is ordered alphabetically.
“List all films in the database by release year” ->: “List all films in the database from a specific release year.” The output is ordered alphabetically.
“List all film by genre” ->: “List all films in the database by specific genres.” The output is ordered alphabetically.
“List all film by genre in the database” ->: “List all films in the database that are a certain genre.” The output is ordered alphabetically.
“List all
Result: Retrieve the details of films with a rating higher than a certain value. ->:
Result: Retrieve the details of films released after a certain year. ->: 1. Create a table c

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [ ]:
output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 120 #500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_dict['train'],
    # train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    # dataset_text_field="prediction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.728400
2,2.528500
3,2.279500
4,2.112600
5,1.790400
6,1.577400
7,1.280500
8,1.188200
9,1.185400
10,1.023300


TrainOutput(global_step=120, training_loss=0.3030404057043294, metrics={'train_runtime': 601.734, 'train_samples_per_second': 3.191, 'train_steps_per_second': 0.199, 'total_flos': 3729307297962240.0, 'train_loss': 0.3030404057043294, 'epoch': 26.67})

In [ ]:
lst_test_data = list(test_df['text'])

In [ ]:
len(lst_test_data)

18

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

0 Retrieve the details of films with their release years converted to Julian day numbers. ->: SELECT film_name, julianday(release_year) AS julian_release_year FROM films; ->: SELECT film_name, julian(release_year) AS julian_release_year FROM films; ->: SELECT film_name, julianday(release_year) AS julian_release_year FROM films WHERE release_year IS NOT
1 Show films with ratings ranked in descending order. ->: SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films; ->: SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films ORDER BY rating DESC; ->: SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films
2 Retrieve the details of films with a rating higher than the average rating of films released in the same year as 'The Matrix'. ->: SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT release_year FROM films WHERE film_name = 'The Matrix')); ->: SELEC

In [ ]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['SELECT film_name, julianday(release_year) AS julian_release_year FROM films;',
 'SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;',
 "SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT release_year FROM films WHERE film_name = 'The Matrix'));",
 'SELECT MAX(rating) FROM films;',
 'SELECT film_name, LENGTH(review) AS review_length FROM films;',
 'SELECT * FROM films;',
 'SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 400 = 0);',
 "SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';: SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';",
 'SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;',
 'SELECT release_year, AVG(rating) FROM films GROUP BY release_year;',
 "SELECT * FROM films WHERE review LIKE '%great%';",
 'SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year > 2000);',
 

In [ ]:
df_evaluate = test_df[['description','query']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['query_predicted'] = answers

df_evaluate

,description,query,query_predicted
0,Retrieve the details of films with their release years converted to Julian day numbers.,"SELECT film_name, julianday(release_year) AS julian_release_year FROM films;","SELECT film_name, julianday(release_year) AS julian_release_year FROM films;"
1,Show films with ratings ranked in descending order.,"SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;","SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;"
2,Retrieve the details of films with a rating higher than the average rating of films released in ...,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT r...,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT r...
3,Retrieve the highest rating among all films.,SELECT MAX(rating) FROM films;,SELECT MAX(rating) FROM films;
4,Display films with their review lengths calculated.,"SELECT film_name, LENGTH(review) AS review_length FROM films;","SELECT film_name, LENGTH(review) AS review_length FROM films;"
5,List all films in the database.,SELECT * FROM films;,SELECT * FROM films;
6,Retrieve films with release years that are leap years.,SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 40...,SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 40...
7,Find the average rating of films watched in 2022.,"SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';","SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';: SELECT AVG(rating) F..."
8,Retrieve the details of films with a rating within a specific range.,SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;,SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;
9,Calculate the average rating of films released in each year.,"SELECT release_year, AVG(rating) FROM films GROUP BY release_year;","SELECT release_year, AVG(rating) FROM films GROUP BY release_year;"


**USING TRAINED MODEL**

In [ ]:
trainer.save_model('/content/trained model')

In [ ]:
trained_model = '/content/trained model'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    trained_model,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(trained_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    lst_test_data,
    max_length=100,  #200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Settin

0 Retrieve the details of films with their release years converted to Julian day numbers. ->: SELECT film_name, julianday(release_year) AS julian_release_year FROM films; ->: SELECT film_name, julianday(release_time) AS julian_release_time FROM films WHERE release_time IS NOT NULL;
The
julianday function takes a string representing the release year and returns the Julian day number corresponding to
1 Show films with ratings ranked in descending order. ->: SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films; ->: SELECT film_name, rating, rating * -1 AS diff FROM films; ->: SELECT film_name, rating, abs(rating - rating_rank) AS distance FROM films; ->: SELECT film_name, rating, COALESCE(rating,
2 Retrieve the details of films with a rating higher than the average rating of films released in the same year as 'The Matrix'. ->: SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT release_year FROM films WHERE film_

In [ ]:
def correct_answer(ans):
  return (ans.split("->:")[1]).strip()

answers = []
for ix,seq in enumerate(sequences):
    # print(ix,seq[0]['generated_text'])
    answers.append(correct_answer(seq[0]['generated_text']))

answers

['SELECT film_name, julianday(release_year) AS julian_release_year FROM films;',
 'SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;',
 "SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT release_year FROM films WHERE film_name = 'The Matrix'));",
 'SELECT MAX(rating) FROM films;',
 'SELECT film_name, LENGTH(review) AS review_length FROM films;',
 'SELECT * FROM films; SELECT film_name, COUNT(*) FROM films GROUP BY film_name; SELECT film_name, COUNT(*) FROM films GROUP BY film_name ORDER BY COUNT(*) DESC LIMIT 10;',
 'SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 400 = 0);',
 "SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';",
 'SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;',
 'SELECT release_year, AVG(rating) FROM films GROUP BY release_year;',
 "SELECT * FROM films WHERE review LIKE '%great%';",
 'SELECT * FROM films WHERE

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
df_evaluate = test_df[['description','query']]

df_evaluate = df_evaluate.reset_index(drop=True)

df_evaluate['query_predicted'] = answers

df_evaluate

,description,query,query_predicted
0,Retrieve the details of films with their release years converted to Julian day numbers.,"SELECT film_name, julianday(release_year) AS julian_release_year FROM films;","SELECT film_name, julianday(release_year) AS julian_release_year FROM films;"
1,Show films with ratings ranked in descending order.,"SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;","SELECT film_name, rating, RANK() OVER (ORDER BY rating DESC) AS rating_rank FROM films;"
2,Retrieve the details of films with a rating higher than the average rating of films released in ...,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT r...,SELECT * FROM films WHERE rating > (SELECT AVG(rating) FROM films WHERE release_year = (SELECT r...
3,Retrieve the highest rating among all films.,SELECT MAX(rating) FROM films;,SELECT MAX(rating) FROM films;
4,Display films with their review lengths calculated.,"SELECT film_name, LENGTH(review) AS review_length FROM films;","SELECT film_name, LENGTH(review) AS review_length FROM films;"
5,List all films in the database.,SELECT * FROM films;,"SELECT * FROM films; SELECT film_name, COUNT(*) FROM films GROUP BY film_name; SELECT film_name,..."
6,Retrieve films with release years that are leap years.,SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 40...,SELECT * FROM films WHERE release_year % 4 = 0 AND (release_year % 100 != 0 OR release_year % 40...
7,Find the average rating of films watched in 2022.,"SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';","SELECT AVG(rating) FROM films WHERE strftime('%Y', watched_time) = '2022';"
8,Retrieve the details of films with a rating within a specific range.,SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;,SELECT * FROM films WHERE rating BETWEEN 3.0 AND 4.5;
9,Calculate the average rating of films released in each year.,"SELECT release_year, AVG(rating) FROM films GROUP BY release_year;","SELECT release_year, AVG(rating) FROM films GROUP BY release_year;"


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
trainer.push_to_hub("gopeshravi/sql-tuned-falcon")

adapter_model.safetensors:   0%|          | 0.00/522M [00:00<?, ?B/s]

events.out.tfevents.1707762753.01bc08640693.380.0:   0%|          | 0.00/24.1k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/gopeshravi/results/commit/a96921e681435cadc4e3d6cd1ea5e1ddef3e8033', commit_message='gopeshravi/sql-tuned-falcon', commit_description='', oid='a96921e681435cadc4e3d6cd1ea5e1ddef3e8033', pr_url=None, pr_revision=None, pr_num=None)

**script version**

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarro

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
import transformers
# from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

trained_model = '/content/trained model'

model = AutoModelForCausalLM.from_pretrained(
    trained_model,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(trained_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)
def query_gen(input: str):
    sequences = pipeline(
        input,
        max_length=100,  #200,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    return seq[0]['generated_text']

# for ix,seq in enumerate(sequences):
#     print(ix,seq[0]['generated_text'])

while(True):
    usr_query = input("Enter query:")
    if usr_query != 'quit':
        sql_query = query_gen(usr_query)
        print(sql_query)
    else:
        break